In [52]:
import pandas as pd
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/challenge.csv")
city_data_df.loc[city_data_df["City"]=="Jamestown"]

,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Jamestown,42.1,-79.24,48.0,clear sky,0.0,0.0,24,1,16.11,US,2020-02-23 19:32:43


In [53]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining on your trip? (yes/no) "))
snow = str(input("Do you want it to be snowing on your trip? (yes/no) "))

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining on your trip? (yes/no) no
Do you want it to be snowing on your trip? (yes/no) no


In [54]:
wantsRain = False
wantsSnow = False
if((rain == 'yes') & (snow == "yes")):
    
    # Filter the dataset for rain and  snow
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) &\
                                           (city_data_df["Rain inches (last 3 hrs)"] >=0) &\
                                           (city_data_df["Snow inches (last 3 hrs)"] >=0)]
#rain and no snow
elif ((rain == 'yes') & (snow == "no")):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) &\
                                       (city_data_df["Rain inches (last 3 hrs)"] >=0) &\
                                       (city_data_df["Snow inches (last 3 hrs)"] ==0)]
#no rain and  snow
elif ((rain == 'no') & (snow == "yes")):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) &\
                                       (city_data_df["Rain inches (last 3 hrs)"] ==0) &\
                                       (city_data_df["Snow inches (last 3 hrs)"] >=0)]
else:
   preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) &\
                                       (city_data_df["Rain inches (last 3 hrs)"] ==0) &\
                                       (city_data_df["Snow inches (last 3 hrs)"] ==0)] 

preferred_cities_df.head()
                                                 


,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.19,168.86,52.00,broken clouds,0.0,0.0,92,77,4.00,NZ,2020-02-23 19:39:18
2,2,Jamestown,42.10,-79.24,48.00,clear sky,0.0,0.0,24,1,16.11,US,2020-02-23 19:32:43
3,3,Smithers,54.78,-127.17,26.60,broken clouds,0.0,0.0,68,75,2.24,CA,2020-02-23 19:39:56
6,6,Zdvinsk,54.70,78.66,9.77,broken clouds,0.0,0.0,99,72,2.15,RU,2020-02-23 19:40:02
9,9,Klaksvik,62.23,-6.59,37.40,scattered clouds,0.0,0.0,64,33,23.04,FO,2020-02-23 19:34:59


In [55]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,broken clouds,-46.19,168.86,
2,Jamestown,US,clear sky,42.10,-79.24,
3,Smithers,CA,broken clouds,54.78,-127.17,
6,Zdvinsk,RU,broken clouds,54.70,78.66,
9,Klaksvik,FO,scattered clouds,62.23,-6.59,
11,Cape Town,ZA,clear sky,-33.93,18.42,
13,Ushuaia,AR,light rain,-54.80,-68.30,
14,Busselton,AU,broken clouds,-33.65,115.33,
15,Esperance,AU,overcast clouds,-33.87,121.90,
17,Castro,BR,overcast clouds,-24.79,-50.01,


In [56]:
import gmaps
import requests
# Import the API key.
from config import g_key
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a marks of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [58]:
hotel_df

,City,Country,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,broken clouds,-46.19,168.86,Ellie's Villa
2,Jamestown,US,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
3,Smithers,CA,broken clouds,54.78,-127.17,Prestige Hudson Bay Lodge
6,Zdvinsk,RU,broken clouds,54.70,78.66,Hotel Siberia
9,Klaksvik,FO,scattered clouds,62.23,-6.59,Airbnb rooms
...,...,...,...,...,...,...
547,Bereda,ES,clear sky,43.27,-7.54,Pazo de Verdes
548,Sept-Iles,CA,broken clouds,50.20,-66.38,Comfort Inn
552,Kikwit,CD,broken clouds,-5.04,18.82,Centre D'Accueil Protestant
553,Bireun,ID,overcast clouds,5.20,96.70,OYO 2303 Hotel Blang Raya


In [59]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")